In [1]:
#########Importing libraries##########
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import  Image
import itertools
import warnings
warnings.filterwarnings("ignore")
import io

############Visualization#############
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
######################################

In [51]:
########Bring in the DATA!###########

telcom = pd.read_csv("db/telemarker_db.csv")
telcom.head()
print(telcom.dtypes)
#######Data Cleaning Process#########

#Replacing spaces with null values in total charges column
telcom['TotalCharges'] = telcom["TotalCharges"].replace(" ",np.nan)
print("Percentage of null: " + str(round(telcom['TotalCharges'].isnull().sum()/telcom['TotalCharges'].count()*100,4))+"%")

#Dropping null values from total charges column which contain .16% missing data
telcom = telcom[telcom["TotalCharges"].notnull()]
telcom = telcom.reset_index()[telcom.columns]

#convert to float type
telcom["TotalCharges"] = telcom["TotalCharges"].astype(float)
telcom["MonthlyCharges"] = telcom["MonthlyCharges"].astype(float)

#replace 'No phone service' to No
telcom["MultipleLines"] = telcom["MultipleLines"].replace("No phone service","No")

#replace 'No internet service' to No for the following columns
replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in replace_cols : 
    telcom[i]  = telcom[i].replace({'No internet service' : 'No'})

#Change types of columns to boolean
YesNoColumns = ["Partner","Dependents","PhoneService","MultipleLines","OnlineSecurity",
                "OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies",
                "PaperlessBilling","Churn"]
for i in YesNoColumns :
    telcom[i]  = telcom[i].replace({'Yes':1,'No':0})
BoolColumns = YesNoColumns.copy()
BoolColumns.append("SeniorCitizen")
telcom[BoolColumns] = telcom[BoolColumns].astype('bool')
telcom
#Separating churn and non churn customers
# churn     = telcom[telcom["Churn"] == "Yes"]
# not_churn = telcom[telcom["Churn"] == "No"]

#     #Separating catagorical and numerical columns
# Id_col     = ['customerID']
# target_col = ["Churn"]
# cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
# cat_cols   = [x for x in cat_cols if x not in target_col]
# num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

customerID           object
gender               object
City                 object
State                object
Phone                object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
Name                 object
Address              object
dtype: object
Percentage of null: 0.1564%


customerID           object
gender               object
City                 object
State                object
Phone                object
SeniorCitizen          bool
Partner                bool
Dependents             bool
tenure                int64
PhoneService           bool
MultipleLines          bool
InternetService      object
OnlineSecurity         bool
OnlineBackup           bool
DeviceProtection       bool
TechSupport            bool
StreamingTV            bool
StreamingMovies        bool
Contract             object
PaperlessBilling       bool
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                  bool
Name                 object
Address              object
dtype: object

In [4]:
#labels
lab = telcom["Churn"].value_counts().keys().tolist()
#values
val = telcom["Churn"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(title = "Customer attrition in data",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data = [trace]
fig = go.Figure(data = data,layout = layout)
py.iplot(fig)